# tensorboard 可视化操作

In [6]:
import tensorflow as tf
a = tf.constant([1.0,2.0,3.0],name='input1')
b = tf.Variable(tf.random_uniform([3]),name='input2')
add = tf.add_n([a,b],name='addOP')
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter("graphs/tfboard_test",sess.graph)
    print(sess.run(add))
writer.close()

[1.4057752 2.8760526 3.6929045]


cmd命令：
tensorboard --logdir ="E:\jupyter notebook\graphs\tfboard_test"

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
import numpy as np
import tensorflow as tf
import time

import utils

# Define paramaters for the model
learning_rate = 0.01
batch_size = 128
n_epochs = 30

# Step 1: Read in data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_DATA', one_hot = True)

# Step 2: Create placeholders for features and labels
X = tf.placeholder(tf.float32, [batch_size, 784], name='X_placeholder')
Y= tf.placeholder(tf.float32, [batch_size, 10], name='Y_placeholder')

# Step 3: create weights and bias
w = tf.Variable(tf.random_normal(shape=[784, 10], stddev= 0.01), name='weights')
b = tf.Variable(tf.zeros([1, 10]), name="bias")

# Step 4: build model
y_predicted = tf.matmul(X, w) + b

# Step 5: define loss function
entropy = tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits = y_predicted, name='loss')
loss = tf.reduce_mean(entropy)

#可视化尝试  
tf.summary.scalar("loss", loss)
tf.summary.histogram("histogram_loss", loss)
summary_op = tf.summary.merge_all()

# Step 6: define training op
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

with tf.Session() as sess:
    # to visualize using TensorBoard
    writer = tf.summary.FileWriter('graphs/tfboard_test',sess.graph)
    
    start_time = time.time()
    sess.run(tf.global_variables_initializer())
    n_batches = int(mnist.train.num_examples/batch_size)
    # train the model n_epochs times
    for i in range(n_epochs):
        total_loss = 0
        
        for _ in range(n_batches):
            X_batch, Y_batch = mnist.train.next_batch(batch_size)
            #_, loss_batch = sess.run([optimizer, loss], feed_dict={X: X_batch, Y:Y_batch})
            _, loss_batch, summary = sess.run([optimizer, loss, summary_op], feed_dict={X: X_batch, Y:Y_batch})
            total_loss += loss_batch
            ##调用writer的add_summary方法将训练过程以及训练步数保存  
            writer.add_summary(summary, global_step = i)
        print ('Average loss epoch {0}: {1}'.format(i, total_loss/n_batches))
        
    print ('Total time: {0} seconds'.format(time.time() - start_time))
    
    #should be around 0.35 after 25 epochs
    print('Optimization Finished!')
    
    #test the model
    n_batches = int(mnist.test.num_examples/batch_size)
    total_correct_preds = 0
    for i in range(n_batches):
        X_batch, Y_batch = mnist.test.next_batch(batch_size)
        _, loss_batch, logits_batch = sess.run([optimizer, loss, y_predicted], feed_dict={X: X_batch, Y: Y_batch})
        preds = tf.nn.softmax(logits_batch)
        correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(Y_batch, 1))
        accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))
        total_correct_preds += sess.run(accuracy)
        
    print ('Accuracy {0}'.format(total_correct_preds/mnist.test.num_examples))

    writer.close()


Instructions for updating:
Use the retry module or similar alternatives.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_DATA\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_DATA\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_DATA\t10k-images-idx3-ubyte.gz
Extracting MNIST_DATA\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

Average loss epoch 0: 1.289507107023314